In [73]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate
from tensorflow.keras.optimizers import Adam

%matplotlib inline

In [200]:
files = os.listdir("ultrasound-nerve-segmentation/train")
image_names = []
for f in files:
    if '_mask' not in f:
        image_names.append(os.path.splitext(f)[0])
        
train_names, test_names = train_test_split(image_names[:600], test_size=0.1)
train_len = len(train_names)
test_len = len(test_names)

In [201]:
img_width = 128
img_height = 128
batch_size = 8
learning_rate = 1e-5

In [202]:
# a smoothed version of dice coefficient
def dice(y, y_pred):
    intersection = K.sum(K.flatten(y) * K.flatten(y_pred))
    return (2.0 * intersection + 1) / (K.sum(y) + K.sum(y_pred) + 1)

def dice_loss(y, y_pred):
    return -tf.log(dice(y, y_pred))

In [208]:
# Build U-Net/FCN style model
def build_net(img_width, img_height, batch_size, learning_rate):
    input_images = Input(shape=(img_width,img_height,1), name='input')
    conv1 = Conv2D(32, (3,3), activation='relu', padding='same')(input_images)
    conv1 = Conv2D(32, (3,3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(2)(conv1)

    conv2 = Conv2D(64, (3,3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3,3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(2)(conv2)

    conv3 = Conv2D(128, (3,3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3,3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(2)(conv3)

    conv4 = Conv2D(256, (3,3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3,3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(2)(conv4)

    conv5 = Conv2D(512, (3,3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3,3), activation='relu', padding='same')(conv5)

    conv6 = concatenate([Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(conv5), conv4])
    conv6 = Conv2D(256, (3,3), activation='relu', padding='same')(conv6)
    conv6 = Conv2D(256, (3,3), activation='relu', padding='same')(conv6)

    conv7 = concatenate([Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(conv6), conv3])
    conv7 = Conv2D(128, (3,3), activation='relu', padding='same')(conv7)
    conv7 = Conv2D(128, (3,3), activation='relu', padding='same')(conv7)

    conv8 = concatenate([Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(conv7), conv2])
    conv8 = Conv2D(64, (3,3), activation='relu', padding='same')(conv8)
    conv8 = Conv2D(64, (3,3), activation='relu', padding='same')(conv8)

    conv9 = concatenate([Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(conv8), conv1])
    conv9 = Conv2D(32, (3,3), activation='relu', padding='same')(conv9)
    conv9 = Conv2D(32, (3,3), activation='relu', padding='same')(conv9)

    output = Conv2D(1, (1,1), activation='sigmoid')(conv9)

    model = Model(inputs=[input_images], outputs=[output])
    model.compile(optimizer=Adam(lr=learning_rate), loss=dice_loss, metrics=[dice])
    
    return model

In [209]:
def preprocess(im):
    im_arr = np.asarray(im.resize((img_width, img_height)))
    return im_arr

def augment(img, label):
    #flip h
    flipped_h = np.flip(img, axis=0)
    flipped_h_lb = np.flip(label, axis=0)
    #flip v
    flipped_v = np.flip(img, axis=1)
    flipped_v_lb = np.flip(label, axis=1)
    #rotate
    rotate_90 = np.rot90(img)
    rotate_90_lb = np.rot90(label)
    rotate_180 = np.rot90(rotate_90)
    rotate_180_lb = np.rot90(rotate_90_lb)
    rotate_270 = np.rot90(rotate_180)
    rotate_270_lb = np.rot90(rotate_180_lb)
    aug_ims = np.array([flipped_h, flipped_v, rotate_90, rotate_180, rotate_270])
    aug_lbs = np.array([flipped_h_lb, flipped_v_lb, rotate_90_lb, rotate_180_lb, rotate_270_lb])
    return (aug_ims, aug_lbs)
    

In [210]:
def get_batches(batch_size, train=True):
    filenames = None
    if train: filenames = train_names
    else: filenames = test_names
    while True:
        for i in range(0, len(filenames), batch_size):
            images = []
            labels = []
            for img_name in filenames[i:i+batch_size]:
                im = Image.open("ultrasound-nerve-segmentation/train/"+img_name+".tif")
                lb = Image.open("ultrasound-nerve-segmentation/train/"+img_name+"_mask.tif")
                im_arr = preprocess(im)
                lb_arr = preprocess(lb)
                images.append(im_arr)
                labels.append(lb_arr)
                
                if train:
                    aug_ims, aug_lbs = augment(im_arr, lb_arr)
                    for im in aug_ims:
                        images.append(im)
                    for lb in aug_lbs:
                        labels.append(lb)
            ims = np.expand_dims(np.array(images, dtype='float32'), axis=3)
            lbls = np.expand_dims(np.array(labels, dtype='float32'), axis=3)

            mean = np.mean(ims)
            std = np.std(ims)
            ims -= mean
            ims /= std
            lbls /= 255.0

            yield ims, lbls


In [213]:
import math
def train(epochs):
    
    gen = get_batches(batch_size)
    val = get_batches(batch_size, train=False)
    model = build_net(img_width, img_height, batch_size, learning_rate)
    checkpoint = ModelCheckpoint('model_weights.hd5', monitor='val_loss')
    model.fit_generator(gen, epochs=epochs, steps_per_epoch=math.ceil(train_len/batch_size),
                        validation_data=val, validation_steps=math.ceil(test_len/batch_size), 
                        verbose=1, callbacks=[checkpoint])
  

In [214]:
train(10)

Epoch 1/10
68/68 [==============================] - 751s 11s/step - loss: 4.0784 - dice: 0.0213 - val_loss: 3.7201 - val_dice: 0.0253
Epoch 2/10
68/68 [==============================] - 761s 11s/step - loss: 4.0360 - dice: 0.0222 - val_loss: 3.5968 - val_dice: 0.0286
Epoch 3/10
68/68 [==============================] - 798s 12s/step - loss: 3.4657 - dice: 0.0414 - val_loss: 2.4301 - val_dice: 0.0913
Epoch 4/10
45/68 [==================>...........] - ETA: 4:23 - loss: 2.6669 - dice: 0.0784

KeyboardInterrupt: 